In [15]:
import re

# Given list
words = ["PROPOS'", 'NATUREJ', 'INGREDIENT MENAGER', 'MULTI-USAGE', 'TERRE dE', 'SOMMIERES', '400% NATUREL', 'Argile', '10036', 'pure', 'et   naturelle', 'Lerre', 'Sommierespreserte des proprietes absorbantes', 'qui permettent', 'nelloyage', 'sec des laches', 'recalcitrantes Sur toutes les surfaces (moquette;', 'tapis; parquel_-', 'Elle', 'est aussi', 'efficace', 'desodoriser', 'linge:', 'Ingredient', 'Bentonite', 'Dosage conseille', 'Sclon usage.', 'S[0an', 'Lunponpacel', 'cetl{ I 0t inunei4', 'Wzouul', 'Rtiu fos;', 'Lnfonr', '00%', '@500 g', 'Jusuac', 'LABORATOIRE RROPOSNATURE', 'UCUGRLDEC . [uacoJi JHH4E 04', 'e+raaly', '\'Mp"0pr-nJinecon', '780 103', 'pour', "'Pnz"]

# 1. Convert the entire list into a single word with no spaces and no commas
single_word = ''.join(words).replace(' ', '').replace(',', '')

# 2. Detect numbers and extract characters around them
hq = []
# Regular expression to find numbers with two characters before and after
pattern = re.compile(r'(.{2})\d+(.{2})')

matches = pattern.finditer(single_word)

for match in matches:
    before = match.group(1)
    number = match.group(0)[2:-2]  # Extract only the number part
    after = match.group(2)
    new_word = before + number + after
    hq.append(new_word)

# Print the results
print("Single Word:", single_word)
print("HQ List:", hq)


Single Word: PROPOS'NATUREJINGREDIENTMENAGERMULTI-USAGETERREdESOMMIERES400%NATURELArgile10036pureetnaturelleLerreSommierespresertedesproprietesabsorbantesquipermettentnelloyagesecdeslachesrecalcitrantesSurtouteslessurfaces(moquette;tapis;parquel_-Elleestaussiefficacedesodoriserlinge:IngredientBentoniteDosageconseilleSclonusage.S[0anLunponpacelcetl{I0tinunei4WzouulRtiufos;Lnfonr00%@500gJusuacLABORATOIRERROPOSNATUREUCUGRLDEC.[uacoJiJHH4E04e+raaly'Mp"0pr-nJinecon780103pour'Pnz
HQ List: ['ES400%N', 'le10036pu', 'S[0an', '{I0ti', 'ei4Wz', 'nr00%@', '500gJ', 'HH4E0', 'p"0pr', 'on780103po']


In [16]:
def extract_and_format(input_list):
    output_list = []
    
    for item in input_list:
        # Extract the numeric part and any following characters
        numeric_part = ''.join(filter(str.isdigit, item))
        non_numeric_part = ''.join(filter(lambda x: not x.isdigit(), item))
        
        # Combine the numeric part with the non-numeric part
        formatted_string = numeric_part + non_numeric_part
        output_list.append(formatted_string)
    
    return ', '.join(output_list)

# Get the formatted output
output = extract_and_format(hq)
print(output)

400ES%N, 10036lepu, 0S[an, 0{Iti, 4eiWz, 00nr%@, 500gJ, 40HHE, 0p"pr, 780103onpo


In [ ]:
import spacy
from spacy.pipeline import EntityRuler
import re
import ast

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Create custom entity ruler
ruler = EntityRuler(nlp)

# Define patterns for weight, volume, wattage, and voltage
patterns = [
    {"label": "WEIGHT", "pattern": [{"LOWER": {"REGEX": "^[0-9,.]+$"}}, {"LOWER": {"IN": ["g", "kg", "oz", "lbs", "gram", "grams", "kilogram", "kilograms", "ounce", "ounces", "pound", "pounds"]}}]},
    {"label": "VOLUME", "pattern": [{"LOWER": {"REGEX": "^[0-9,.]+$"}}, {"LOWER": {"IN": ["ml", "l", "oz", "fl oz", "milliliter", "milliliters", "liter", "liters", "ounce", "ounces", "fluid ounce", "fluid ounces"]}}]},
    {"label": "WATTAGE", "pattern": [{"LOWER": {"REGEX": "^[0-9,.]+$"}}, {"LOWER": "w"}]},
    {"label": "VOLTAGE", "pattern": [{"LOWER": {"REGEX": "^[0-9,.]+$"}}, {"LOWER": "v"}]}
]

# Add patterns to the ruler
ruler.add_patterns(patterns)

# Add the ruler to the pipeline
nlp.add_pipe("entity_ruler", config={"overwrite_ents": True})

def extract_entity(text, entity_type):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == entity_type:
            value = re.findall(r'[\d,.]+', ent.text)[0]
            unit = re.findall(r'[a-zA-Z]+', ent.text)[0]
            return f"{float(value.replace(',', ''))} {unit}"
    return None

# Read the text snippets and entity types
with open('extracted_text.txt', 'r') as f:
    text_content = f.read()
    text_snippets = ast.literal_eval(text_content)

with open('entity_name.txt', 'r') as f:
    entity_types = [line.strip() for line in f if line.strip()]

# Process each text snippet and extract the relevant entity
for text_list, entity_type in zip(text_snippets, entity_types):
    text = ' '.join(text_list)  # Join the list of words into a single string
    if entity_type == "item_weight":
        result = extract_entity(text, "WEIGHT")
    elif entity_type == "item_volume":
        result = extract_entity(text, "VOLUME")
    elif entity_type == "wattage":
        result = extract_entity(text, "WATTAGE")
    elif entity_type == "voltage":
        result = extract_entity(text, "VOLTAGE")
    else:
        result = None
    
    print(f"Text: {text}")
    print(f"Entity Type: {entity_type}")
    print(f"Extracted Value: {result}")
    print()